In [ ]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
import pandas as pd

from skimage import morphology, measure
from skimage.segmentation import expand_labels, watershed, relabel_sequential
from sklearn.preprocessing import minmax_scale
from scipy import ndimage as ndi
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask import delayed
from dask.distributed import Client, progress
from colorsys import hls_to_rgb

import napari

In [ ]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

In [ ]:
#Specific is the name of the folder of the orginal data
specific = 'F231006L'
#finger_num should be 1,2,3, or 4, corresponding to the index, middle, ring, and pinky fingers
finger_num = 2

These are numbers you ought to adjust after you look at Napari.

In [ ]:
bone_threshold = 300
eig_1_threshold = 74
eig_3_threshold = -54
closing_radius = 18
expand_labels_distance = 15
close_expanded_distance = 15
connect_num = 3

In [ ]:
finger_names = {1: 'index', 2: 'middle', 3: 'ring', 4: 'pinky'}
finger_name = finger_names[finger_num]
digit_names = {1: 'proximal', 2: 'middle', 3: 'distal'}

In [ ]:
full_finger_dir = f'../fingers/{finger_name}/full/{specific}/'
finger_eig_dir = f'../fingers/{finger_name}/full/{specific} - eigens/'
digit_dir = {i: f'../fingers/{finger_name}/{digit_names[i]}/{specific}/' for i in range(1,4)}
digit_eig_dir = {i: f'../fingers/{finger_name}/{digit_names[i]}/{specific} - eigens/' for i in range(1,4)}
temp_finger = 'temp-finger/'
temp_eigs = 'temp-eigs'

In [ ]:
all_dirs = [full_finger_dir, finger_eig_dir, temp_finger, temp_eigs] + list(digit_dir.values()) + list(digit_eig_dir.values())
for d in all_dirs:
    os.makedirs(d, exist_ok=True)

In [ ]:
spacing = np.load('../spacing/' + specific + '.npy')

In [ ]:
finger = da.from_npy_stack(full_finger_dir)

In [ ]:
eigs = da.from_npy_stack(finger_eig_dir)

In [ ]:
shape = finger.shape
shape

In [ ]:
finger_2 = da.swapaxes(
    da.rechunk(finger, chunks = (shape[0], 1, shape[2])),
    0, 1
)
da.to_npy_stack(temp_finger, finger_2)

In [ ]:
finger_2 = da.from_npy_stack(temp_finger)

In [ ]:
#Matrix multiplication functions as the dot product of the left-rows and the right-columns
#The trace is x [1,1,1]
#and the differences will be x[1,-1,0] and x[0,1,-1]
eigs_2 = da.swapaxes(
    da.rechunk(
        eigs,
        chunks = (shape[0], 1, shape[2],1)
    ), 
    0, 1
)
da.to_npy_stack(temp_eigs, eigs_2)

In [ ]:
eigs_2 = da.from_npy_stack(temp_eigs)

In [ ]:
viewer = napari.Viewer()

viewer.add_image(
    finger_2,
    scale = spacing,
)

viewer.add_image(
    eigs_2,
    channel_axis=3,
    name=["eig 1", "eig 2", "eig 3"],
    colormap=["red", "green", "blue"],
    scale = spacing,
)

In [ ]:
eig_1 = eigs_2[:,:,:,0]
eig_3 = eigs_2[:,:,:,2]
enough_bone = da.where(finger_2 > bone_threshold, 1, 0).compute()
enough_eig_1 = da.where(eig_1 > eig_1_threshold, 1, 0).compute()
enough_eig_3 = da.where(eig_3 < eig_3_threshold, 0, 1).compute()


In [ ]:
separated = enough_bone * enough_eig_1 * enough_eig_3
del enough_eig_1, enough_eig_3

Add points here if you need to take away parts where bones touch

In [ ]:
points = viewer.add_points(scale=spacing)

In [ ]:
points.data = np.round(points.data)
for p in points.data:
    a,b,c = (p).astype('int16')
    separated[a,b,c] = 0

In [ ]:
viewer.add_image(
    separated,
    scale = spacing,
    name = 'separated?'
)

In [ ]:
#This is the NumPy/Skimage way

#Use the label function to get all connected components
l_start = measure.label(separated, connectivity=connect_num)
#Count the sizes of the labels
counts = np.bincount(l_start.ravel())
#The indices of the thring digits will be 2nd->proximal, 3rd->middle, 4th->distal (the biggest connected component is the outside)
digit_indices = counts.argsort()[-2:-5:-1]
#Create a NumPy label map sending proximal->1, middle->2, distal->3
label_map = np.zeros(counts.shape, dtype=np.dtype('uint8'))
for new_label, old_label in enumerate(digit_indices, start=1):
    label_map[old_label] = new_label
#Use the label map
l_start_re = label_map[l_start]

#This is the Dask Way
#Incomplete because so far, individual fingers have been small enough for our computers
#label_start, num_labels = ndmeasure.label(separated)

In [ ]:
viewer.add_labels(
    l_start_re,
    scale = spacing,
    name = 'labels started?'
)

In [ ]:
proximal = l_start_re == 1
medial = l_start_re == 2
distal = l_start_re == 3

In [ ]:
del l_start, l_start_re

In [ ]:
p_closed = morphology.closing(
    proximal,
    footprint = morphology.ball(
        closing_radius,
        decomposition = 'sequence',
    )
)
m_closed = morphology.closing(
    medial,
    footprint = morphology.ball(
        closing_radius,
        decomposition = 'sequence',
    )
)
d_closed = morphology.closing(
    distal,
    footprint = morphology.ball(
        closing_radius,
        decomposition = 'sequence',
    )
)

In [ ]:
del distal, medial, proximal,

In [ ]:
l_s_closed = p_closed + 2 * m_closed + 3 * d_closed

In [ ]:
del p_closed, m_closed, d_closed

In [ ]:
viewer.add_labels(
    l_s_closed,
    scale = spacing,
    name = 'closed starting labels',
)

In [ ]:
distance = ndi.distance_transform_edt(separated)
labels = watershed(-distance, l_s_closed, mask=enough_bone)

In [ ]:
del separated, distance

In [ ]:
viewer.add_labels(
    labels,
    scale = spacing,
    name = 'labeled completely?'
)

In [ ]:
expanded = morphology.closing(
    expand_labels(labels, distance = expand_labels_distance),
    footprint = morphology.ball(
        close_expanded_distance,
        decomposition = 'sequence',
    )
)


In [ ]:
del labels

In [ ]:
viewer.add_labels(
    expanded,
    scale = spacing,
    name = 'expanded labels'
)

In [ ]:
for i in range(1,4):
    sub_x, sub_y, sub_z = np.where(expanded == i)
    
    x_min = sub_x.min()
    x_max = sub_x.max()
    del sub_x
    
    y_min = sub_y.min()
    y_max = sub_y.max()
    del sub_y
    
    z_min = sub_z.min()
    z_max = sub_z.max()
    del sub_z
    
    subset = np.where(expanded == i, finger_2.compute(), -1000)[x_min:x_max, y_min:y_max, z_min:z_max]
    da.to_npy_stack(digit_dir[i], subset)

    sub_eigs = np.where(expanded[..., np.newaxis] == i, eigs_2.compute(), -1000)[x_min:x_max, y_min:y_max, z_min:z_max, :]
    da.to_npy_stack(digit_eig_dir[i], sub_eigs)
    
    print(f'x from {x_min} to {x_max}')
    print(f'y from {y_min} to {y_max}')
    print(f'z from {z_min} to {z_max}')
    #print(f'Size of digit {i} array: {size(subset.nbytes)}')
    print('_'*20)